# Libraries

In [21]:
import pandas as pd
import crewai
from crewai import Agent, Task, Crew, Process
import os
from crewai_tools import SerperDevTool
from crewai import LLM
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

# LLM

In [23]:
load_dotenv()
gpt = LLM(
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    top_p=0.1,
    api_key=os.getenv("OPENAI_API_KEY")
)

# Tool

In [24]:
search_tool = SerperDevTool(
    n_results= 3,
    country="th",
    locale="tha",
    location="Thailand, Thai",
)
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

# Agent

In [25]:
research_agent = Agent(
    role="Data Researcher",
    goal="Gather the current price for various laptop and laptop models by searching",
    backstory="A dedicated researcher specializing in extracting product information.",
    tools=[search_tool],
    memory=True,
    verbose=True,
    max_iter=5
)

# Task

In [26]:
data_gathering_task = Task(
    description="""
        Search for the following laptop model "{brand_model}" and collect data on:
        - Current Price (in THB)
        - Links to price information
        
        Ensure the information is accurate and sourced from trusted sites.
        - If there is no available data, mark it as 'Unknown.'
        - Please include links as evidence for each data point.
    """,

    expected_output=""" 
    A JSON object in this format:
        "search_model": {brand_model},
        "laptop_model": "<laptop>",
        "current_price_in_thb": "<current price in THB>" in only numeric format,
        "current_price_link": "<link to source>" 
    """,
    tools=[search_tool],
    agent=research_agent
)

# Crew

In [ ]:
crew = Crew(
    agents=[research_agent],
    tasks=[data_gathering_task],
    process=Process.sequential
)

# Data Example

In [37]:
df = pd.read_csv("laptops_model.csv")
df.head(3)

,Laptops
0,HP Victus 15.6 inch Gaming Laptop 15-fa1093TX
1,ThinkPad E16 G2
2,"Apple 15"" MacBook Air M2"


# Usage

In [28]:
from datetime import datetime
import csv
import time
import random

def load_laptop_data(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header if present
        return [row[0] for row in reader]


csv_file_path = "laptops_model.csv"
laptops_list = load_laptop_data(csv_file_path)

# from row to row
def run_batch(start, end):
    for laptop in laptops_list[start:end]:
        print(f"Gathering data for {laptop}")
        time.sleep(random.uniform(2, 4))
        result = crew.kickoff(inputs={'brand_model': laptop})
        print(result)
        now = datetime.now()
        current = now.strftime("%Y-%m-%d %H:%M:%S")
        print(current)

In [ ]:
run_batch(0,101)